In [1]:

import json
import openai
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import pandas as pd
from langchain.chat_models import ChatOpenAI
import json
from IPython.display import display, Markdown
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### Load and Inspect Raw Claims

This step reads the JSON file containing raw expense claim entries and prints each claim ID to confirm the file was loaded correctly.


In [15]:

def read_and_print_claim_ids(file_path):
    """
    Reads a JSON file containing claims and prints the claim IDs.
    """
    try:
        with open(file_path, "r") as file:
            claims_data = json.load(file)
        
        # Print each claim ID
        for claim_id in claims_data.keys():
            print(f"Found Claim ID: {claim_id}")
            
    except Exception as e:
        print(f"Error reading JSON file: {str(e)}")


read_and_print_claim_ids("Data/raw_claims.json")

Found Claim ID: claim_001
Found Claim ID: claim_002
Found Claim ID: claim_003
Found Claim ID: claim_004
Found Claim ID: claim_005
Found Claim ID: claim_006
Found Claim ID: claim_007
Found Claim ID: claim_008
Found Claim ID: claim_009
Found Claim ID: claim_010
Found Claim ID: claim_011
Found Claim ID: claim_012
Found Claim ID: claim_013
Found Claim ID: claim_014
Found Claim ID: claim_015
Found Claim ID: claim_016
Found Claim ID: claim_017
Found Claim ID: claim_018
Found Claim ID: claim_019
Found Claim ID: claim_020
Found Claim ID: claim_021
Found Claim ID: claim_022
Found Claim ID: claim_023
Found Claim ID: claim_024
Found Claim ID: claim_025
Found Claim ID: claim_026
Found Claim ID: claim_027
Found Claim ID: claim_028
Found Claim ID: claim_029
Found Claim ID: claim_030
Found Claim ID: claim_031
Found Claim ID: claim_032
Found Claim ID: claim_033
Found Claim ID: claim_034
Found Claim ID: claim_035
Found Claim ID: claim_036
Found Claim ID: claim_037
Found Claim ID: claim_038
Found Claim 

### Process and Structure Claims Using OpenAI

This function reads raw claim text, sends it to an LLM for structured extraction, and appends it to the appropriate DataFrame (`meal_df`, `travel_df`, or `unidentified_df`) based on the claim type.

### From Natural Text Claims to Structured Claim Objects

This step processes unstructured text claims submitted by claimants and converts them into **structured claim objects**. The goal is to ensure the claims can be easily validated against the directive's rules. 


---

###  **Process Overview**
1. **Input:** Raw text describing a claim, either a travel or meal expense.
2. **Processing:** The text is sent to a language model (LLM) that analyzes the content and categorizes it as either a **Meal Claim** or a **Travel Claim**. 
3. **Output:** A structured claim object (JSON format) that contains specific attributes required for validation.

---

###  **Structure of Claim Objects**

#### Meal Claim Object

```json
{
    "claim_type": "meal",
    "is_government_business": true,
    "is_away_from_office_area": true,
    "prior_approval_obtained": true,
    "meal_purchased": true,
    "alcohol_included": false,
    "within_allowable_rates": true,
    "itemized_receipt_provided": true,
    "is_group_meal": false,
    "claimed_by_senior_person": false
}
```


### Travel Claim Object

```json
{
    "claim_type": "travel",
    "is_government_business": true,
    "is_regular_commute": false,
    "travel_pre_approved": true,
    "timely_submission": true,
    "documentation_provided": true,
    "travel_type": "air/rail",
    "economy_class": true,
    "business_class_approved": false,
    "vehicle_approved": false,
    "accommodation_approved": false
}
```
These objects contain a simplified version of a claim, in the actual application; the fields;
```json
{
"is_government_business",
"within_allowable_rates",
"itemized_receipt_provided",
"claimed_by_senior_person"
}
```
would need to be calculated using: location, OCR (for scanning receipts), lookup for senior person




### **Example Conversion Process**

#### Natural Text Claim (Submitted by Claimant)
> "I traveled for government business to attend a conference. The travel was pre-approved, and all the documentation was provided. I traveled by air in economy class, and I submitted the claim within the required period."

---

#### **Converted Structured Claim Object (Travel Claim)**
```json
{
    "claim_type": "travel",
    "is_government_business": true,
    "is_regular_commute": false,
    "travel_pre_approved": true,
    "timely_submission": true,
    "documentation_provided": true,
    "travel_type": "air/rail",
    "economy_class": true,
    "business_class_approved": false,
    "vehicle_approved": false,
    "accommodation_approved": false
}


In [3]:
def process_claim_text(claim_text):
    """
    Takes a single claim text as input, sends it to the LLM, 
    and returns the structured JSON response.
    """
    prompt = f"""
    You are an intelligent assistant that reads expense claims and extracts structured information.
    The claim may be a meal or a travel expense. Return the information as a JSON object.

    For meal claims, provide:
    {{
        "claim_type": "meal",
        "is_government_business": true/false,
        "is_away_from_office_area": true/false,
        "prior_approval_obtained": true/false,
        "meal_purchased": true/false,
        "alcohol_included": true/false,
        "within_allowable_rates": true/false,
        "itemized_receipt_provided": true/false,
        "is_group_meal": true/false,
        "claimed_by_senior_person": true/false
    }}

    For travel claims, provide:
    {{
        "claim_type": "travel",
        "is_government_business": true/false,
        "is_regular_commute": true/false,
        "travel_pre_approved": true/false,
        "timely_submission": true/false,
        "documentation_provided": true/false,
        "travel_type": "air/rail" / "vehicle" / "accommodation",
        "economy_class": true/false,
        "business_class_approved": true/false,
        "vehicle_approved": true/false,
        "accommodation_approved": true/false
    }}

    Process the following claim text and return the structured information.

    Claim Text:
    {claim_text}
    """
    model = ChatOpenAI(temperature=0)
    response = model.predict(prompt)
    
    try:
        structured_data = json.loads(response)
        return structured_data
    except json.JSONDecodeError:
        return {"error": "Failed to parse response as JSON"}




### **Testing conversion from Claim Text to Claim Object**
#### Claim Text:
"I purchased lunch during a business trip 50 km away from the office. The meal was pre-approved, no alcohol was included, and I provided an itemized receipt, the cost is within the limits."


In [ ]:
claim_text = "I purchased lunch during a business trip 50 km away from the office. The meal was pre-approved, no alcohol was included, and I provided an itemized receipt and the cost is within the limits."

test_obj = process_claim_text(claim_text)
display(Markdown(f"```json\n{json.dumps(test_obj, indent=4)}\n```"))
#below line must be executed after the validate methods are executed
#print("Validation Result : ", validate_claim(test_obj))


```json
{
    "claim_type": "meal",
    "is_government_business": true,
    "is_away_from_office_area": true,
    "prior_approval_obtained": true,
    "meal_purchased": true,
    "alcohol_included": false,
    "within_allowable_rates": true,
    "itemized_receipt_provided": true,
    "is_group_meal": false,
    "claimed_by_senior_person": false
}
```

Validation Result :  {'status': 'Approved', 'reason': 'Within allowable rates'}


### Initializing DataFrames for Claims Storage

This step creates empty DataFrames to store structured claims data. Each type of claim (meal, travel, unidentified) has its own DataFrame with specific columns corresponding to the extracted attributes.


In [5]:

# Creating empty DataFrames for each type of claim
meal_df = pd.DataFrame(columns=[
    "Claim ID", "Claim Type", "Is Government Business", "Is Away From Office Area",
    "Prior Approval", "Meal Purchased", "Alcohol Included",
    "Within Allowable Rates", "Itemized Receipt Provided",
    "Is Group Meal", "Claimed by Senior Person"
])

travel_df = pd.DataFrame(columns=[
    "Claim ID", "Claim Type", "Is Government Business", "Is Regular Commute",
    "Travel Pre-Approved", "Timely Submission", "Documentation Provided",
    "Travel Type", "Economy Class", "Vehicle Approved", "Accommodation Approved"
])

unidentified_df = pd.DataFrame(columns=["Claim ID", "Claim Text", "Reason"])


### Process Raw Claims and Populate DataFrames

This function reads claim entries from a JSON file, processes each one using the `process_claim_text` function, and adds the structured output to the appropriate DataFrame:

- **Meal claims** → `meal_df`
- **Travel claims** → `travel_df`
- **Invalid/unrecognized claims** → `unidentified_df`



In [6]:
def process_json_claims(file_path):
    """
    Reads claims from a JSON file and applies process_claim_text to each claim.
    Writes the processed claims to the appropriate DataFrame.
    """
    global meal_df, travel_df, unidentified_df  # Referencing the DataFrames defined above

    with open(file_path, "r") as file:
        claims_data = json.load(file)

    for claim_id, claim_info in claims_data.items():
        claim_text = claim_info["claim_text_content"]
        structured_data = process_claim_text(claim_text)
        
        if structured_data.get("error"):
            # Log unidentified claim
            unidentified_df = pd.concat([unidentified_df, pd.DataFrame([{
                "Claim ID": claim_id,
                "Claim Text": claim_text,
                "Reason": structured_data["error"]
            }])], ignore_index=True)
            #print(f"Unidentified Claim: {claim_id} - {structured_data['error']}")
        
        elif structured_data["claim_type"] == "meal":
            # Add claim to the meal DataFrame
            meal_row = {
                "Claim ID": claim_id,
                "Claim Type": structured_data.get("claim_type", ""),
                "Is Government Business": structured_data.get("is_government_business", ""),
                "Is Away From Office Area": structured_data.get("is_away_from_office_area", ""),
                "Prior Approval": structured_data.get("prior_approval_obtained", ""),
                "Meal Purchased": structured_data.get("meal_purchased", ""),
                "Alcohol Included": structured_data.get("alcohol_included", ""),
                "Within Allowable Rates": structured_data.get("within_allowable_rates", ""),
                "Itemized Receipt Provided": structured_data.get("itemized_receipt_provided", ""),
                "Is Group Meal": structured_data.get("is_group_meal", ""),
                "Claimed by Senior Person": structured_data.get("claimed_by_senior_person", "")
            }
            filtered_row = {key: meal_row[key] for key in meal_df.columns}
            meal_df = pd.concat([meal_df, pd.DataFrame([filtered_row])], ignore_index=True)
            #print(f"Meal Claim Processed: {claim_id}")

        elif structured_data["claim_type"] == "travel":
            # Add claim to the travel DataFrame
            travel_row = {
                "Claim ID": claim_id,
                "Claim Type": structured_data.get("claim_type", ""),
                "Is Government Business": structured_data.get("is_government_business", ""),
                "Is Regular Commute": structured_data.get("is_regular_commute", ""),
                "Travel Pre-Approved": structured_data.get("travel_pre_approved", ""),
                "Timely Submission": structured_data.get("timely_submission", ""),
                "Documentation Provided": structured_data.get("documentation_provided", ""),
                "Travel Type": structured_data.get("travel_type", ""),
                "Economy Class": structured_data.get("economy_class", ""),
                "Vehicle Approved": structured_data.get("vehicle_approved", ""),
                "Accommodation Approved": structured_data.get("accommodation_approved", "")
            }
            filtered_row = {key: travel_row[key] for key in travel_df.columns}
            travel_df = pd.concat([travel_df, pd.DataFrame([filtered_row])], ignore_index=True)
            #print(f"Travel Claim Processed: {claim_id}")

        else:
            # Log unidentified claim if the type is not recognized
            unidentified_df = pd.concat([unidentified_df, pd.DataFrame([{
                "Claim ID": claim_id,
                "Claim Text": claim_text,
                "Reason": "Claim type not recognized."
            }])], ignore_index=True)
            #print(f"Unidentified Claim: {claim_id} - Claim type not recognized.")


### Validate Travel Claims Against Directive Rules

This function applies decision logic to a structured travel claim dictionary. It checks for compliance with government business rules, pre-approval, documentation, and conditions specific to the travel type (air/rail, vehicle, or accommodation), returning an appropriate approval or rejection message.


In [7]:
def validate_travel_claim(claim):
    if not claim["is_government_business"]:
        return {"status": "Rejected", "reason": "Not for government business"}
    
    if claim["is_regular_commute"]:
        return {"status": "Rejected", "reason": "Regular commute not covered"}
    
    if not claim["travel_pre_approved"]:
        return {"status": "Rejected", "reason": "Travel not pre-approved"}
    
    if not claim["timely_submission"]:
        return {"status": "Rejected", "reason": "Late submission"}
    
    if not claim["documentation_provided"]:
        return {"status": "Rejected", "reason": "Documentation not provided"}
    
    travel_type = claim["travel_type"]
    
    if travel_type in ("air/rail", "air", "rail"):
        if claim["economy_class"] or claim["business_class_approved"]:
            return {"status": "Approved", "reason": "Approved air/rail travel"}
        else:
            return {"status": "Rejected", "reason": "Air/rail travel not approved"}
    
    elif travel_type == "vehicle":
        if claim["vehicle_approved"]:
            return {"status": "Approved", "reason": "Approved vehicle travel"}
        else:
            return {"status": "Rejected", "reason": "Vehicle not approved"}
    
    elif travel_type == "accommodation":
        if claim["accommodation_approved"]:
            return {"status": "Approved", "reason": "Approved accommodation"}
        else:
            return {"status": "Rejected", "reason": "Accommodation not approved"}
    
    return {"status": "Rejected", "reason": "Invalid travel claim"}


### Validate Meal Claims Against Directive Rules

This function checks whether a structured meal claim meets all directive requirements. It evaluates government business relevance, distance from office, prior approval, alcohol exclusion, rate limits, and group meal conditions. The output is an approval or a specific rejection reason based on the decision path.


In [8]:
def validate_meal_claim(claim):
    if not claim["is_government_business"]:
        return {"status": "Rejected", "reason": "Not for government business"}
    
    if not claim["is_away_from_office_area"] and not claim["prior_approval"]:
        return {"status": "Rejected", "reason": "Must be 24 km away or approved"}
    
    if not claim["meal_purchased"]:
        return {"status": "Rejected", "reason": "Meals must be purchased"}
    
    if claim["alcohol_included"]:
        return {"status": "Rejected", "reason": "Alcohol not reimbursable"}
    
    if claim["within_allowable_rates"]:
        return {"status": "Approved", "reason": "Within allowable rates"}
    else:
        if not claim["itemized_receipt_provided"]:
            return {"status": "Rejected", "reason": "Itemized receipt required"}
    
    if claim["is_group_meal"]:
        if not claim["claimed_by_senior_person"]:
            return {"status": "Rejected", "reason": "Only senior person claims group meals"}
    else:
        return {"status": "Rejected", "reason": "Exceeds allowable rates & not group meal"}
    
    return {"status": "Approved", "reason": "Exceeds allowable rates group meal"}


### Unified Claim Validation Dispatcher

This function routes a structured claim to the appropriate validator: `validate_meal_claim` or `validate_travel_claim` based on its type (`meal` or `travel`). It acts as a central entry point for applying the decision logic to any processed claim.


In [9]:
def validate_claim(claim):
    """
    Validate a claim by determining if it's a meal or travel claim
    and passing it to the appropriate validation function.
    """
    if claim["claim_type"] == "meal":
        return validate_meal_claim(claim)
    elif claim["claim_type"] == "travel":
        return validate_travel_claim(claim)
    else:
        return "Invalid Claim Type"

In [ ]:
claim_text = "I purchased lunch during a business trip 50 km away from the office. The meal was pre-approved, no alcohol was included, and I provided an itemized receipt and the cost is within the limits."

test_obj = process_claim_text(claim_text)
display(Markdown(f"```json\n{json.dumps(test_obj, indent=4)}\n```"))
#below line must be executed after the validate methods are executed
print("Validation Result : ", validate_claim(test_obj))


```json
{
    "claim_type": "meal",
    "is_government_business": true,
    "is_away_from_office_area": true,
    "prior_approval_obtained": true,
    "meal_purchased": true,
    "alcohol_included": false,
    "within_allowable_rates": true,
    "itemized_receipt_provided": true,
    "is_group_meal": false,
    "claimed_by_senior_person": false
}
```

Validation Result :  {'status': 'Approved', 'reason': 'Within allowable rates'}


### Validate All Claims in a DataFrame

This function loops through each row of a DataFrame containing structured claims, maps the fields to the expected dictionary format, and applies the appropriate validation logic. It appends a new column, `Validation Result`, to indicate whether each claim is approved or rejected.


In [10]:
def validate_claims_in_df(df):
    statuses = []
    reasons = []

    for index, row in df.iterrows():
        claim_type = row["Claim Type"]
        claim_id = row["Claim ID"]

        if claim_type == "meal":
            claim_data = {
                "claim_type": "meal",
                "is_government_business": row.get("Is Government Business", False),
                "is_away_from_office_area": row.get("Is Away From Office Area", False),
                "prior_approval": row.get("Prior Approval", False),
                "meal_purchased": row.get("Meal Purchased", False),
                "alcohol_included": row.get("Alcohol Included", False),
                "within_allowable_rates": row.get("Within Allowable Rates", False),
                "itemized_receipt_provided": row.get("Itemized Receipt Provided", False),
                "is_group_meal": row.get("Is Group Meal", False),
                "claimed_by_senior_person": row.get("Claimed by Senior Person", False)
            }
            result = validate_meal_claim(claim_data)

        elif claim_type == "travel":
            claim_data = {
                "claim_type": "travel",
                "is_government_business": row.get("Is Government Business", False),
                "is_regular_commute": row.get("Is Regular Commute", False),
                "travel_pre_approved": row.get("Travel Pre-Approved", False),
                "timely_submission": row.get("Timely Submission", False),
                "documentation_provided": row.get("Documentation Provided", False),
                "travel_type": row.get("Travel Type", ""),
                "economy_class": row.get("Economy Class", False),
                "business_class_approved": row.get("Business Class Approved", False),
                "vehicle_approved": row.get("Vehicle Approved", False),
                "accommodation_approved": row.get("Accommodation Approved", False)
            }
            result = validate_travel_claim(claim_data)
        else:
            result = {"status": "Rejected", "reason": "Invalid claim type"}
        
        statuses.append(result["status"])
        reasons.append(result["reason"])
        print(f"Validation result for Claim ID {claim_id}: {result['status']} - {result['reason']}")

    df["Validation Status"] = statuses
    df["Validation Reason"] = reasons

    return df


### Run Claim Processing and Validation

This step triggers the full pipeline:

- **`process_json_claims()`** reads raw claims, uses the LLM to extract structured data, and routes claims into `meal_df`, `travel_df`, or `unidentified_df`.

- **`validate_claims_in_df()`** runs validation logic on each structured claim by calling either `validate_meal_claim()` or `validate_travel_claim()`, and adds a `Validation Result` column.

### **Cost and Duration**
1 claim is around 300 tokens with input output and instructions
total of 1M claims is equal to 300M tokens, at around 1$/1M tokens in/out the cost is ~300$
Time taken per claim is ~1 second ~280 hours

In [11]:
process_json_claims("Data/raw_claims.json")
travel_df = validate_claims_in_df(travel_df)
meal_df = validate_claims_in_df(meal_df)


Validation result for Claim ID claim_001: Approved - Approved air/rail travel
Validation result for Claim ID claim_003: Rejected - Travel not pre-approved
Validation result for Claim ID claim_005: Rejected - Late submission
Validation result for Claim ID claim_006: Rejected - Travel not pre-approved
Validation result for Claim ID claim_007: Rejected - Travel not pre-approved
Validation result for Claim ID claim_009: Rejected - Documentation not provided
Validation result for Claim ID claim_011: Rejected - Late submission
Validation result for Claim ID claim_013: Rejected - Travel not pre-approved
Validation result for Claim ID claim_014: Rejected - Late submission
Validation result for Claim ID claim_016: Rejected - Late submission
Validation result for Claim ID claim_017: Approved - Approved accommodation
Validation result for Claim ID claim_020: Rejected - Travel not pre-approved
Validation result for Claim ID claim_021: Rejected - Air/rail travel not approved
Validation result for C

### Export Processed Claims to Excel

This function writes the `meal_df`, `travel_df`, and `unidentified_df` DataFrames including the **validation results** to separate sheets in a single Excel file for reporting and review.


In [12]:
def save_dfs_to_excel(meal_df, travel_df, unidentified_df, file_name="Results/processed_claims.xlsx"):
    with pd.ExcelWriter(file_name) as writer:
        meal_df.to_excel(writer, sheet_name="Meal Claims", index=False)
        travel_df.to_excel(writer, sheet_name="Travel Claims", index=False)
        unidentified_df.to_excel(writer, sheet_name="Unidentified Claims", index=False)
    
    print(f"DataFrames have been successfully saved to {file_name}")

save_dfs_to_excel(meal_df, travel_df, unidentified_df)


DataFrames have been successfully saved to Results/processed_claims.xlsx
